# Pipeline Research
when developing a pipeline, I use this notebook to play around with the data and see what I can do with it.

In [1]:
from pipeline.keyword_extractor import DefaultKeywordExtractor
from datasets.docs_dataset import DocsDataset
from datasets.stock_dataset import StockMeta
from pipeline.docs_filterer import DefaultFilterer
from pipeline.docs_labeler import DefaultDocsLabeler

doc_dataset = DocsDataset(documents_csv_path="./organized_data/documents.csv")
stock_meta = StockMeta(stock_meta_path="./organized_data/stock_metadata.csv")

filterer = DefaultFilterer()
labeler = DefaultDocsLabeler(s=3)
keyword_extractor = DefaultKeywordExtractor()

In [2]:
stock_name = '台積電'
stock = stock_meta.get_stock_by_name(stock_name)

In [3]:
filtered_docs = filterer.filter_documents(doc_dataset, stock)

In [4]:
labeled_docs = labeler.label_documents(filtered_docs, stock)

In [5]:
labeled_keyword_docs = keyword_extractor.extract_keywords(labeled_docs)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\break\AppData\Local\Temp\jieba.cache
Loading model cost 0.510 seconds.
Prefix dict has been built successfully.


KeyboardInterrupt: 

In [ ]:
doc_dataset[0]

In [ ]:
import jieba.analyse

doc = doc_dataset[100]
doc_str = ";".join([doc.title, doc.author, doc.content])
# extract the top 10 keywords
keywords = jieba.analyse.extract_tags(doc_str, topK=30, withWeight=True)

# print the keywords and their weights
print('Keywords:')
for keyword, weight in keywords:
    print(keyword, weight)

# extract the top 5 topics
topics = jieba.analyse.textrank(doc_str, topK=20, withWeight=True)

# print the topics and their weights
print('Topics:')
for topic, weight in topics:
    print(topic, weight)